# Solving nonlinear Elasticity

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw

a rectangle with refinement at corners:

In [ ]:
from netgen.occ import *
shape = Rectangle(1,0.1).Face()
shape.edges.Max(X).name="right"
shape.edges.Min(X).name="left"
shape.edges.Max(Y).name="top"
shape.edges.Min(Y).name="bot"
shape.vertices.Min(X+Y).maxh=0.01
shape.vertices.Min(X-Y).maxh=0.01
mesh = Mesh(OCCGeometry(shape, dim=2).GenerateMesh(maxh=0.05))

Cauchy-Green tensor

$$
C = F^T F \qquad \text{with} \qquad F = I + \nabla u
$$

and hyperelastic energy density

$$
W : {\mathbb R}^{d \times d, sym} \rightarrow {\mathbb R}
$$

In [ ]:
E, nu = 210, 0.2
mu  = E / 2 / (1+nu)
lam = E * nu / ((1+nu)*(1-2*nu))

def C(u):
    F = Id(2) + Grad(u)
    return F.trans * F

def NeoHooke (C):
    return 0.5*mu*(Trace(C-Id(2)) + 2*mu/lam*Det(C)**(-lam/2/mu)-1)

stationary point of total energy:

$$
\delta \int W(C(u)) - f u = 0
$$

In [ ]:
factor = Parameter(0)
force = CoefficientFunction( (0,factor) )

fes = H1(mesh, order=4, dirichlet="left", dim=mesh.dim)
u  = fes.TrialFunction()

a = BilinearForm(fes, symmetric=True)
a += Variation(NeoHooke(C(u)).Compile()*dx)
a += Variation((-InnerProduct(force,u)).Compile()*dx)

gfu = GridFunction(fes)
gfu.vec[:] = 0

The `Variation` function declares that the non-linear form is the derivative of the energy.

a simple Newton solver, using automatic differentiation for residual and tangential stiffness:

In [ ]:
def SolveNewton(printrates=False):
    for it in range(10):
        if (printrates):
            print ("it", it, "energy = ", a.Energy(gfu.vec))
        res = a.Apply(gfu.vec)
        a.AssembleLinearization(gfu.vec)
        inv = a.mat.Inverse(fes.FreeDofs() ) 
        gfu.vec.data -= inv*res

In [ ]:
factor.Set(0.4)
SolveNewton(printrates=True)
scene = Draw (C(gfu)[0,0]-1, mesh, deformation=gfu, min=-0.1, max=0.1)

Often, we don't have a good starting value for Newton's method. This can be overcome by increasing the load step by step (assuming the solution depends continuously on the loading). The solution of the previous load-step is the initial guess for the next step.  

In [ ]:
numsteps = 5
maxload = 2
for ls in range (numsteps):
    factor.Set(maxload*(ls+1)/numsteps)
    SolveNewton()
    Draw (C(gfu)[0,0]-1, mesh, deformation=gfu, min=-0.2, max=0.2)

## Stress tensor

Compute $2^{nd}$ Piola Kirchhoff stress tensor by symbolic differentiation:

$$
\Sigma_{i,j} = \frac{\partial W}{\partial C_{i,j}} (C)
$$

In [ ]:
C_=C(gfu).MakeVariable()
sigma = NeoHooke(C_).Diff(C_)

Draw (sigma[0,0], mesh, "Sxx", deformation=gfu, min=-10.001, max=10.001); 

The energy functional is represented as an expression tree:

In [ ]:
u  = fes.TrialFunction()
print (NeoHooke(C(u)))

With the `Compile` method, the tree is linearized, and common sub-expressions are merged:

In [ ]:
print (NeoHooke(C(u)).Compile())

In [ ]:
print (sigma)

In [ ]:
print (sigma.Compile())